# Task 4: Time-delay embedding

This notebook deals with Task 4 of Sheet 5


## Part 1


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.signal import argrelextrema


# Import functions
from models.time_delay import create_time_delay_embedding, analyze_embedding_quality, TimeDelayEmbedding


In [ ]:
# Load the periodic data
data = np.loadtxt('../data/periodic.txt')
print(f"Data shape: {data.shape}")
print(f"First few rows:\n{data[:5]}")

In [ ]:
x = data[:, 0]
y = data[:, 1]

# Plot the original 2D
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(x, y, 'b-', linewidth=1)
plt.xlabel('$x_0$')
plt.ylabel('$x_1$')
plt.title('First column ($x_0$) vs Second column ($x_1$)')
plt.grid(True)
plt.axis('equal')

# Plot the first coordinate against time (row number)
plt.subplot(1, 2, 2)
time_indices = np.arange(len(x))
plt.plot(time_indices, y, 'r-', linewidth=1)
plt.xlabel('time')
plt.ylabel('$x_0$')
plt.title('First column ($x_0$) vs Time (index)')
plt.grid(True)

plt.tight_layout()
plt.show()

### Taken's theorem 

According to Taken's theorem, for d-dimensions, we need at least 2d+1 coordinates for a proper embedding. Since our manifold is one-dimensional (d=1), we need at least (2*1) + 1 = 3 coordinates.

In [ ]:
delays = [10, 50, 100, 150]  # Different delay values
embed_dim = 3

fig, axes = plt.subplots(len(delays), 3, figsize=(20, 5 * len(delays)))

for row, delay in enumerate(delays):
    try:
        # Create the embedding
        embedded = create_time_delay_embedding(x, delay, embed_dim)
        
        # Plot 1: 2D projection ($x_0$(t) vs $x_0$(t + τ))
        ax1 = axes[row, 0]
        ax1.plot(embedded[:, 0], embedded[:, 1], 'b-', linewidth=1, alpha=0.8)
        ax1.set_xlabel('$x_0$(t)')
        ax1.set_ylabel(f'$x_0$(t + τ), τ = {delay}')
        ax1.set_title(f'$x_0$(t) and $x_0$(t + τ) in 2D space, τ = {delay}')
        ax1.grid(True, alpha=0.3)
        ax1.axis('equal')
        
        # Plot 2: Time series showing all three coordinates
        ax2 = axes[row, 1]
        time_subset = np.arange(len(embedded))
        ax2.plot(time_subset, embedded[:, 0], 'b-', linewidth=1, label='$x_0$(t)', alpha=0.8)
        ax2.plot(time_subset, embedded[:, 1], 'r-', linewidth=1, label=f'$x_0$(t + {delay})', alpha=0.8)
        ax2.plot(time_subset, embedded[:, 2], 'g-', linewidth=1, label=f'$x_0$(t + 2τ)', alpha=0.8)
        ax2.set_xlabel('t (line number)')
        ax2.set_ylabel('$x_0$(t), $x_0$(t + τ) and $x_0$(t + 2τ)')
        ax2.set_title(f'$x_0$(t), $x_0$(t + τ) and $x_0$(t + 2τ) against t, τ = {delay}, 2τ = {2*delay}')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # Plot 3: 3D embedding
        ax3 = axes[row, 2]
        ax3 = fig.add_subplot(len(delays), 3, row * 3 + 3, projection='3d')
        ax3.plot(embedded[:, 0], embedded[:, 1], embedded[:, 2], 'g-', linewidth=1, alpha=0.8)
        ax3.set_xlabel('$x_0$(t)')
        ax3.set_ylabel(f'$x_0$(t + τ)')
        ax3.set_zlabel(f'$x_0$(t + 2τ)')
        ax3.set_title(f'$x_0$(t), $x_0$(t + τ) and $x_0$(t + 2τ) in 3D space, 2τ = {2*delay}')
        
    except ValueError as e:
        # Handle errors for any subplot
        for col in range(3):
            ax = axes[row, col]
            ax.text(0.5, 0.5, f'Error: {str(e)}', transform=ax.transAxes, 
                   ha='center', va='center', fontsize=8)
            ax.set_title(f'Delay τ = {delay} - Error')

plt.tight_layout()
plt.show()

### Optimal Tau

In [ ]:

def get_optimal_tau(x, maxtau=100, plot=True):
    """
    Compute the optimal delay (τ) for time-delay embedding using Average Mutual Information (AMI).
    
    Parameters:
        x       : 1D numpy array (time series data)
        maxtau  : maximum delay to check
        plot    : whether to plot the AMI curve
    
    Returns:
        optimal_tau : first local minimum of AMI curve (int), or None if not found
    """
    #mutual_info calculation from numpy
    amis = []
    x_series = np.array(x)
    
    for tau in range(1, maxtau + 1):
        # Calculate mutual information for this tau
        x1 = x_series[:-tau] if tau > 0 else x_series
        x2 = x_series[tau:] if tau > 0 else x_series
        
        # Simple mutual information estimation using histograms
        bins = min(int(np.sqrt(len(x1))), 30)  # Reasonable bin count
        hist_2d, _, _ = np.histogram2d(x1, x2, bins=bins)
        hist_2d = hist_2d / np.sum(hist_2d)  # Normalize
        
        px = np.sum(hist_2d, axis=1)  # p(x)
        py = np.sum(hist_2d, axis=0)  # p(y)
        
        pxy = hist_2d.flatten()
        px_py = np.outer(px, py).flatten()
        
        # Avoid log(0)
        indices = (pxy > 0) & (px_py > 0)
        mutual_info = np.sum(pxy[indices] * np.log(pxy[indices] / px_py[indices]))
        amis.append(mutual_info)
    
    tau_range = np.arange(1, maxtau + 1)

    if plot:
        plt.plot(tau_range, amis)
        plt.xlabel("Delay τ")
        plt.ylabel("Average Mutual Information")
        plt.title("AMI vs Delay")
        plt.grid(True)
        plt.show()

    # Find local minima
    amis_array = np.array(amis)
    minima_indices = argrelextrema(amis_array, np.less)[0]

    if len(minima_indices) > 0:
        optimal_tau = minima_indices[0] + 1  # +1 because AMI starts from τ=1
        return optimal_tau
    else:
        print("No local minimum found in AMI curve.")
        return None
    
# Create single example visualization with optimal delay
tau = get_optimal_tau(x, maxtau=100, plot=True)
print(f"Optimal delay τ = {tau}")


The optimal delay is the first local minimum

In [ ]:

# Use the calculated optimal delay 
delay = tau 
embedded_3d = create_time_delay_embedding(x, delay, 3)

fig = plt.figure(figsize=(18, 6))
# Plot 1: x vs y (original 2D trajectory)
ax1 = fig.add_subplot(131)
ax1.plot(x, y, 'b-', linewidth=1, alpha=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Original Trajectory: x vs y')
ax1.grid(True)
ax1.axis('equal')

# Plot 2: 2D projection of embedding
ax2 = fig.add_subplot(132)
ax2.plot(embedded_3d[:, 0], embedded_3d[:, 1], 'r-', linewidth=1, alpha=0.8)
ax2.set_xlabel('$x_0$(t)')
ax2.set_ylabel(f'$x_0$(t + {delay})')
ax2.set_title(f'2D Projection: $x_0$(t) vs $x_0$(t + {delay})')
ax2.grid(True)
ax2.axis('equal')

# Plot 3: 3D time delay embedding
ax3 = fig.add_subplot(133, projection='3d')
ax3.plot(embedded_3d[:, 0], embedded_3d[:, 1], embedded_3d[:, 2], 'g-', linewidth=1, alpha=0.8)
ax3.set_xlabel('$x_0$(t)')
ax3.set_ylabel(f'$x_0$(t + {delay})')
ax3.set_zlabel(f'$x_0$(t + {2*delay})')
ax3.set_title('3D Time Delay Embedding')

plt.tight_layout()
plt.show()

### Analysis of Embedding Quality (Optimal dimension)

In [ ]:
# Analyze embedding quality
delays_test = [4,5,tau,7,8]  # Convert single delay to list for the function
embed_dims_test = [2, 3, 4, 5, 6, 7, 8, 9, 10]  # Test a range of embedding dimensions

quality_results = analyze_embedding_quality(x, delays_test, embed_dims_test)
print("Embedding Quality Analysis:")
print(quality_results.head(10))

In [ ]:
# Plot quality metrics
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Average correlation vs delay for different embedding dimensions
for embed_dim in embed_dims_test:
    subset = quality_results[quality_results['embed_dim'] == embed_dim]
    axes[0].plot(subset['delay'], subset['avg_correlation'], 'o-', label=f'Dim {embed_dim}')

axes[0].set_xlabel('Delay')
axes[0].set_ylabel('Average Correlation')
axes[0].set_title('Average Correlation Between Dimensions')
axes[0].legend()
axes[0].grid(True)

# Plot 2: Total variance vs delay
for embed_dim in embed_dims_test:
    subset = quality_results[quality_results['embed_dim'] == embed_dim]
    axes[1].plot(subset['delay'], subset['total_variance'], 's-', label=f'Dim {embed_dim}')

axes[1].set_xlabel('Delay')
axes[1].set_ylabel('Total Variance')
axes[1].set_title('Total Variance in Embedding')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Part 2

In [ ]:
from systems.system import LorenzSystem
from scipy.integrate import solve_ivp


In [ ]:
#Create Lorenz system instance
lorenz = LorenzSystem(sigma=10, rho=28, beta=8/3)
initial_state = np.array([10.0, 10.0, 10.0])

# Define the ODE for scipy
def lorenz_ode(t, state):
    return lorenz._get_tangent(t, state)

# Simulate
t_span = [0, 50]
t_eval = np.arange(0, 50, 0.01)
# Solve the Lorenz system ODE
sol = solve_ivp(lorenz_ode, t_span, initial_state, t_eval=t_eval, method='RK45')

x_signal = sol.y[0, :]  # Only x-coordinate
time = sol.t


# Time-delay embedding parameters
delay_steps = 10  # Number of time steps for delay

# Create time-delay coordinates
# x₁ = x(t), x₂ = x(t + Δn), x₃ = x(t + 2Δn)
n_points = len(x_signal)
max_delay = 2 * delay_steps  # For 3D embedding

# Create the embedded coordinates
x1 = x_signal[0:n_points-max_delay]              # x(t)
x2 = x_signal[delay_steps:n_points-delay_steps] # x(t + Δn)
x3 = x_signal[2*delay_steps:n_points]           # x(t + 2Δn)

# 4. Visualize the results
fig = plt.figure(figsize=(18, 6))

# Original x-coordinate time series
ax1 = fig.add_subplot(131)
ax1.plot(time[:1000], x_signal[:1000], 'b-', linewidth=1)
ax1.set_xlabel('Time')
ax1.set_ylabel('x(t)')
ax1.set_title('Original x-coordinate time series')
ax1.grid(True)

# 2D embedding - x(t) vs x(t + Δn)
ax2 = fig.add_subplot(132)
ax2.plot(x1, x2, 'r-', linewidth=0.5, alpha=0.7)
ax2.set_xlabel('x(t)')
ax2.set_ylabel(f'x(t + Δn; Δn= {delay_steps})')
ax2.set_title('2D Time-Delay Embedding')
ax2.grid(True)

# 3D embedding - x(t) vs x(t + Δn) vs x(t + 2Δn)
ax3 = fig.add_subplot(133, projection='3d')
ax3.plot(x1, x2, x3, 'g-', linewidth=0.5, alpha=0.7)
ax3.set_xlabel('x(t)')
ax3.set_ylabel(f'x(t + Δn; Δn= {delay_steps})')
ax3.set_zlabel(f'x(t + 2Δn; 2Δn= {2*delay_steps})')
ax3.set_title('3D Time-Delay Embedding\n(Reconstructed Attractor)')

plt.tight_layout()
plt.show()


In [ ]:

# Test different delays to show the effect

delays_to_test = [5, 10, 20, 40]
time = time[:len(x_signal)] 

n_delays = len(delays_to_test)
fig = plt.figure(figsize=(18, 6 * n_delays))

for i, delay_steps in enumerate(delays_to_test):
    max_delay = 2 * delay_steps
    if len(x_signal) <= max_delay:
        continue

    # Create time-delay coordinates
    x1 = x_signal[0:len(x_signal) - max_delay]              # x(t)
    x2 = x_signal[delay_steps:len(x_signal) - delay_steps]  # x(t + Δn)
    x3 = x_signal[2 * delay_steps:len(x_signal)]            # x(t + 2Δn)

    # Plot index offset for each row
    row = i

    # Original time series
    ax1 = fig.add_subplot(n_delays, 3, row * 3 + 1)
    ax1.plot(time[:1000], x_signal[:1000], 'b-', linewidth=1)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('x(t)')
    ax1.set_title(f'Delay = {delay_steps})\nOriginal Time Series')
    ax1.grid(True)

    # 2D embedding
    ax2 = fig.add_subplot(n_delays, 3, row * 3 + 2)
    ax2.plot(x1, x2, 'r-', linewidth=0.5, alpha=0.7)
    ax2.set_xlabel('x(t)')
    ax2.set_ylabel(f'x(t + Δn)')
    ax2.set_title('2D Time-Delay Embedding')
    ax2.grid(True)

    # 3D embedding
    ax3 = fig.add_subplot(n_delays, 3, row * 3 + 3, projection='3d')
    ax3.plot(x1, x2, x3, 'g-', linewidth=0.5, alpha=0.7)
    ax3.set_xlabel('x(t)')
    ax3.set_ylabel('x(t + Δn)')
    ax3.set_zlabel('x(t + 2Δn)')
    ax3.set_title('3D Time-Delay Embedding')

plt.tight_layout()
plt.show()

In [ ]:
# Test different delays using Z-coordinates instead of X-coordinates

# Extract z-coordinate from the Lorenz system
z_signal = sol.y[2, :]  # z-coordinate instead of x-coordinate

delays_to_test = [5, 10, 20, 40]
time_z = time[:len(z_signal)] 

n_delays = len(delays_to_test)
fig = plt.figure(figsize=(18, 6 * n_delays))

for i, delay_steps in enumerate(delays_to_test):
    max_delay = 2 * delay_steps
    if len(z_signal) <= max_delay:
        continue

    # Create time-delay coordinates using z-signal
    z1 = z_signal[0:len(z_signal) - max_delay]              # z(t)
    z2 = z_signal[delay_steps:len(z_signal) - delay_steps]  # z(t + Δn)
    z3 = z_signal[2 * delay_steps:len(z_signal)]            # z(t + 2Δn)

    # Plot index offset for each row
    row = i

    # Original z-coordinate time series
    ax1 = fig.add_subplot(n_delays, 3, row * 3 + 1)
    ax1.plot(time_z[:1000], z_signal[:1000], 'b-', linewidth=1)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('z(t)')
    ax1.set_title(f'Delay = {delay_steps} \nOriginal Z-coordinate Time Series')
    ax1.grid(True)

    # 2D embedding using z-coordinates
    ax2 = fig.add_subplot(n_delays, 3, row * 3 + 2)
    ax2.plot(z1, z2, 'r-', linewidth=0.5, alpha=0.7)
    ax2.set_xlabel('z(t)')
    ax2.set_ylabel(f'z(t + Δn)')
    ax2.set_title('2D Time-Delay Embedding (Z-coords)')
    ax2.grid(True)

    # 3D embedding using z-coordinates
    ax3 = fig.add_subplot(n_delays, 3, row * 3 + 3, projection='3d')
    ax3.plot(z1, z2, z3, 'purple', linewidth=0.5, alpha=0.7)
    ax3.set_xlabel('z(t)')
    ax3.set_ylabel('z(t + Δn)')
    ax3.set_zlabel('z(t + 2Δn)')
    ax3.set_title('3D Time-Delay Embedding (Z-coords)')
    ax3.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

